In [1]:
from DatasetHandler.BiwiBrowser import *
from LSTM_VGG16.LSTM_VGG16Helper import *
%matplotlib inline

Using TensorFlow backend.


In [2]:
output_begin = 4
num_outputs = 1

In [3]:
timesteps = 26 # TimeseriesGenerator Handles overlapping
in_epochs = 1
out_epochs = 3
train_batch_size = 4
test_batch_size = 1

In [4]:
subjectList = [1, 2, 3, 4, 5, 7, 8, 11, 12, 14] # except [6, 13, 10, ]
testSubjects = [9]

In [ ]:
num_datasets = len(subjectList)

In [ ]:
def getFinalModel(num_outputs = num_outputs):
    inp = (224, 224, 3) # BIWI_Frame_Shape
    vgg_model = VGG16(weights='imagenet', input_shape = inp) #BIWI_Frame_Shape, include_top=False
    """"
    """
    vgg_model.layers.pop()
    vgg_model.outputs = [vgg_model.layers[-1].output]#
    vgg_model.output_layers = [vgg_model.layers[-1]]#
    vgg_model.layers[-1].outbound_nodes = []#
    nb_pretrained_layers = len(vgg_model.layers)
    for layer in vgg_model.layers: #
        layer.trainable = False#
    #print(nb_pretrained_layers)
    #vgg_model.summary()
    rnn = Sequential()
    rnn.add(TimeDistributed(vgg_model, input_shape=(timesteps, inp[0], inp[1], inp[2]), name = 'tdVGG16')) 
    rnn.add(TimeDistributed(Flatten()))
    
   # rnn.add(TimeDistributed(Dense(4096, activation='relu'), name = 'fc1024')), activation='relu'
   # rnn.add(TimeDistributed(Dense(4096, activation='relu'), name = 'fc104'))    
   # rnn.add(TimeDistributed(Dropout(0.25)))
    rnn.add(TimeDistributed(Dense(1024, activation='relu'), name = 'fc10'))#
    rnn.add(TimeDistributed(Dropout(0.25)))

    rnn.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25))
   # rnn.add(Flatten())
    rnn.add(Dense(num_outputs))
    #print(len(rnn.layers))
    for layer in rnn.layers[:1]:#
        layer.trainable = False#
    adam = optimizers.Adam(lr=0.0001)
    rnn.compile(optimizer=adam, loss='mean_squared_error', metrics=['mae'])
    return rnn

In [ ]:
keras.backend.clear_session()#
full_model = getFinalModel(num_outputs = num_outputs)

446586880/553467096 [=======================>......] - ETA: 2:36

In [ ]:
full_model.summary()

In [ ]:
full_model = trainImageModelForEpochs(full_model, out_epochs, subjectList, testSubjects, timesteps, False, output_begin, num_outputs, batch_size = train_batch_size, in_epochs = in_epochs)

In [ ]:
test_generators, test_labelSets = getTestBiwiForImageModel(testSubjects, timesteps, False, output_begin, num_outputs, batch_size = test_batch_size)
test_gen, test_labels = test_generators[0], test_labelSets[0] #[1]

In [ ]:
predictions = full_model.predict_generator(test_gen, verbose = 1)
#predictions = full_model.predict(test_gen[0][0], verbose = 1)
output1 = numpy.concatenate((test_labels[timesteps:, :1], predictions[:, :1]), axis=1)

In [ ]:
print([i[0] for i in predictions[:10]])

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(output1)

In [ ]:
exit()

In [ ]:
len(test_gen[0][0])

In [ ]:
test_gen[23][0][0][0][124][110]